In [30]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
%rm -rf ./logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [31]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [32]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [33]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.0005, 0.0001]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_LEARNING_RATE, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [34]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

In [35]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [36]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      for learning_rate in HP_LEARNING_RATE.domain.values:
        hparams = {
            HP_NUM_UNITS: num_units,
            HP_DROPOUT: dropout_rate,
            HP_LEARNING_RATE: learning_rate,
            HP_OPTIMIZER: optimizer,
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'learning_rate': 0.0001, 'optimizer': 'adam'}
313/313 [==============================] - 0s 308us/step - loss: 0.5139 - accuracy: 0.8225
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'learning_rate': 0.0005, 'optimizer': 'adam'}
313/313 [==============================] - 0s 305us/step - loss: 0.5056 - accuracy: 0.8222
--- Starting trial: run-2
{'num_units': 16, 'dropout': 0.1, 'learning_rate': 0.001, 'optimizer': 'adam'}
313/313 [==============================] - 0s 309us/step - loss: 0.4822 - accuracy: 0.8294
--- Starting trial: run-3
{'num_units': 16, 'dropout': 0.1, 'learning_rate': 0.0001, 'optimizer': 'sgd'}
313/313 [==============================] - 0s 324us/step - loss: 0.6752 - accuracy: 0.7644
--- Starting trial: run-4
{'num_units': 16, 'dropout': 0.1, 'learning_rate': 0.0005, 'optimizer': 'sgd'}
313/313 [==============================] - 0s 317us/step - loss: 0.6544 - accuracy: 0.7797
--- Starting trial

In [37]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6010 (pid 94041), started 0:43:31 ago. (Use '!kill 94041' to kill it.)